## Config .env
DB_USER=user_securite_routiere
DB_PASSWORD=password123
DB_HOST=localhost
DB_PORT=5432
DB_NAME=securite_routiere_db

Une fois que le fichier .env est bien configuré on peut lancer ça et commencer à travailler

In [ ]:
import sys
sys.path.append("..")  # remonte d'un niveau vers le dossier parent
from utils.utils import connect_to_db, insert_data_to_db, mapping_colonnes, fetch_data_from_db
engine = connect_to_db()
df = fetch_data_from_db(table_name='accidents_cleaned', engine=engine, schema='silver')
df

✅ Connexion à la base PostgreSQL réussie !
OK: "silver"."accidents_cleaned"


,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,year_georef,com_name,dep_code,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,code_com
0,201600055393,2016-03-14 08:20:00+00:00,Paris-17e-arrondissement,2016,3,14,09:20,Plein jour,En agglomération,2,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 17e Arrondissement,75117
1,201600056285,2016-08-03 05:40:00+00:00,Paris-19e-arrondissement,2016,8,3,07:40,Plein jour,En agglomération,2,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 19e Arrondissement,75119
2,201600056621,2016-09-22 10:25:00+00:00,Paris-20e-arrondissement,2016,9,22,12:25,Plein jour,En agglomération,1,...,2016,Paris,75,Paris,200054781,Métropole du Grand Paris,11,Île-de-France,Paris 20e Arrondissement,75120
3,201600057859,2016-09-14 15:15:00+00:00,La Plaine-des-palmistes,2016,9,14,17:15,Plein jour,Hors agglomération,1,...,2016,La Plaine-des-Palmistes,974,La Réunion,249740093,CA Intercommunale de la Réunion Est (Cirest),04,La Réunion,La Plaine-des-Palmistes,97406
4,201400013574,2014-01-27 17:45:00+00:00,Aubagne,2014,1,27,18:45,Nuit avec éclairage public allumé,En agglomération,3,...,2015,Aubagne,13,Bouches-du-Rhône,241300268,CA du Pays d'Aubagne et de l'Etoile (Capae),93,Provence-Alpes-Côte d'Azur,Aubagne,13005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600049674,2016-01-28 03:40:00+00:00,Canejan,2016,1,28,04:40,Nuit sans éclairage public,Hors agglomération,1,...,2016,Canéjan,33,Gironde,243301165,CC Jalle-Eau-Bourde,75,Aquitaine-Limousin-Poitou-Charentes,Canéjan,33090
475907,201600049793,2016-07-13 08:25:00+00:00,Gradignan,2016,7,13,10:25,Plein jour,Hors agglomération,1,...,2016,Gradignan,33,Gironde,243300316,Bordeaux Métropole,75,Aquitaine-Limousin-Poitou-Charentes,Gradignan,33192
475908,201600050415,2016-09-08 05:45:00+00:00,Toulon,2016,9,8,07:45,Plein jour,Hors agglomération,1,...,2016,Toulon,83,Var,248300543,CA Toulon Provence Méditerranée,93,Provence-Alpes-Côte d'Azur,Toulon,83137
475909,201600027375,2016-02-06 23:05:00+00:00,Villeneuve-sur-lot,2016,2,7,00:05,Nuit sans éclairage public,Hors agglomération,3,...,2016,Villeneuve-sur-Lot,47,Lot-et-Garonne,200023307,CA du Grand Villeneuvois,75,Aquitaine-Limousin-Poitou-Charentes,Villeneuve-sur-Lot,47323


In [ ]:
import pandas as pd

df_silver = df.copy()  # Supposons que df_silver est le DataFrame nettoyé après vos étapes de nettoyage.

# =====================================================================
# BLOC DE DIAGNOSTIC COMPLET SUR LE DATAFRAME SILVER (df_silver)
# Objectif : Valider le nettoyage et préparer la couche Gold.
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' in locals() and df_silver is not None:
    print("=======================================================================")
    print("        Diagnostic Complet des Données Nettoyées (df_silver)         ")
    print("=======================================================================")

    # --- 1. Analyse de la complétion (Taux de remplissage) ---
    print("\n--- 1. Taux de Remplissage par Colonne (de la pire à la meilleure) ---")
    
    total_lignes = len(df_silver)
    comptage_non_na = df_silver.count()
    taux_remplissage = (comptage_non_na / total_lignes) * 100
    
    stats_remplissage = pd.DataFrame({'Taux_Remplissage': taux_remplissage})
    stats_remplissage['Taux_Remplissage'] = stats_remplissage['Taux_Remplissage'].map('{:.2f} %'.format)
    print(stats_remplissage.sort_values(by='Taux_Remplissage').to_string())
    
    seuil_suppression = 10.0
    colonnes_a_supprimer = taux_remplissage[taux_remplissage < seuil_suppression].index.tolist()
    if colonnes_a_supprimer:
        print(f"\n💡 Suggestion : Les colonnes suivantes sont remplies à moins de {seuil_suppression}% et pourraient être supprimées avant la couche Gold :")
        print(colonnes_a_supprimer)


    # --- 2. Analyse de la Cardinalité (Nombre de valeurs uniques) ---
    print("\n\n--- 2. Cardinalité (Nombre de Valeurs Uniques) par Colonne ---")
    
    cardinalite = df_silver.nunique()
    stats_cardinalite = pd.DataFrame({'Nombre_Valeurs_Uniques': cardinalite})
    print(stats_cardinalite.sort_values(by='Nombre_Valeurs_Uniques', ascending=False).to_string())

    colonnes_inutiles = cardinalite[cardinalite == 1].index.tolist()
    if colonnes_inutiles:
        print("\n💡 Suggestion : Les colonnes suivantes n'ont qu'une seule valeur et sont probablement inutiles :")
        print(colonnes_inutiles)


    # --- 3. Description Statistique des Colonnes Numériques ---
    print("\n\n--- 3. Description Statistique des Colonnes Numériques ---")
    
    # On sélectionne les colonnes numériques après votre nettoyage
    df_numerique = df_silver.select_dtypes(include=['int64', 'float64', 'Int64', 'Float64'])
    if not df_numerique.empty:
        # .describe() calcule les stats, .round(2) arrondit à 2 décimales.
        print(df_numerique.describe().round(2).to_string())
    else:
        print("Aucune colonne numérique trouvée pour l'analyse.")
            
    print("\n=======================================================================")

else:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé. Veuillez d'abord l'importer et le nettoyer.")

        Diagnostic Complet des Données Nettoyées (df_silver)         

--- 1. Taux de Remplissage par Colonne (de la pire à la meilleure) ---
             Taux_Remplissage
occutc                 1.26 %
v1                    10.56 %
num_veh              100.00 %
grav                 100.00 %
sexe                 100.00 %
catr                 100.00 %
catv                 100.00 %
year_georef          100.00 %
com                  100.00 %
dep                  100.00 %
col                  100.00 %
num_acc              100.00 %
int                  100.00 %
datetime             100.00 %
an                   100.00 %
mois                 100.00 %
jour                 100.00 %
hrmn                 100.00 %
catu                 100.00 %
agg                  100.00 %
lum                  100.00 %
infra                 11.45 %
an_nais               18.95 %
obs                   21.30 %
locp                  21.40 %
etatp                 27.89 %
v2                     4.10 %
date              

In [ ]:
import pandas as pd 

# =====================================================================
# DIAGNOSTIC ET AMÉLIORATION DES COORDONNÉES GPS DANS df_silver
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister en mémoire.
if 'df_silver' not in locals() or df_silver is None:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé en mémoire.")
else:
    print("DataFrame 'df_silver' trouvé. Début du processus d'amélioration des coordonnées GPS...")

    # --- ÉTAPE 1 : Diagnostic Avant Amélioration ---
    # ... (cette partie ne change pas)
    total_lignes = len(df_silver)
    nan_avant_lat = df_silver['lat'].isna().sum()
    pourcentage_avant_lat = (nan_avant_lat / total_lignes) * 100
    print(f"\nAvant amélioration (lat) : {nan_avant_lat:,} valeurs manquantes ({pourcentage_avant_lat:.2f} %).")
    nan_avant_long = df_silver['long'].isna().sum()
    pourcentage_avant_long = (nan_avant_long / total_lignes) * 100
    print(f"Avant amélioration (long): {nan_avant_long:,} valeurs manquantes ({pourcentage_avant_long:.2f} %).")

    # --- ÉTAPE 2 : Remplissage des Coordonnées Manquantes (Logique Corrigée) ---
    # LA CORRECTION EST ICI : On vérifie si 'lat' OU '|' 'long' est vide.
    # Les parenthèses sont importantes pour l'ordre des opérations.
    condition_remplissage = (df_silver['lat'].isna() | df_silver['long'].isna()) & df_silver['coordonnees'].notna()
    
    nb_lignes_a_remplir = condition_remplissage.sum()
    print(f"\nTrouvé {nb_lignes_a_remplir:,} lignes où au moins une coordonnée (lat/long) peut être récupérée.")

    if nb_lignes_a_remplir > 0:
        coords_split = df_silver.loc[condition_remplissage, 'coordonnees'].str.split(',', expand=True)
        
        if coords_split.shape[1] >= 2:
            # On remplit 'lat' UNIQUEMENT si elle est vide
            df_silver.loc[condition_remplissage & df_silver['lat'].isna(), 'lat'] = coords_split[0]
            # On remplit 'long' UNIQUEMENT si elle est vide
            df_silver.loc[condition_remplissage & df_silver['long'].isna(), 'long'] = coords_split[1]
    
            # --- ÉTAPE 3 : Reconversion des Types ---
            df_silver['lat'] = pd.to_numeric(df_silver['lat'], errors='coerce')
            df_silver['long'] = pd.to_numeric(df_silver['long'], errors='coerce')

            # --- ÉTAPE 4 : Diagnostic Après Amélioration ---
            # ... (cette partie ne change pas)
            nan_apres_lat = df_silver['lat'].isna().sum()
            pourcentage_apres_lat = (nan_apres_lat / total_lignes) * 100
            print(f"\nAprès amélioration (lat) : {nan_apres_lat:,} valeurs manquantes ({pourcentage_apres_lat:.2f} %).")
            nan_apres_long = df_silver['long'].isna().sum()
            pourcentage_apres_long = (nan_apres_long / total_lignes) * 100
            print(f"Après amélioration (long): {nan_apres_long:,} valeurs manquantes ({pourcentage_apres_long:.2f} %).")
            
            print("\n✅ Succès ! Le DataFrame 'df_silver' en mémoire a été mis à jour.")
        else:
            print("⚠️ Attention : La découpe de la colonne 'coordonnees' n'a pas produit 2 colonnes. Le remplissage est annulé.")
    else:
        print("\n-> Aucune coordonnée supplémentaire à remplir.")

DataFrame 'df_silver' trouvé. Début du processus d'amélioration des coordonnées GPS...

Avant amélioration (lat) : 233,966 valeurs manquantes (49.16 %).
Avant amélioration (long): 233,966 valeurs manquantes (49.16 %).

Trouvé 218,808 lignes où au moins une coordonnée (lat/long) peut être récupérée.


/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_6431/2035906928.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['-21.152906' '43.331442' '16.257101' ... '45.740286' '43.124011'
 '48.888312']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_silver.loc[condition_remplissage & df_silver['lat'].isna(), 'lat'] = coords_split[0]
/var/folders/4_/cb0dgr6d07d_36m06pf2rqjr0000gn/T/ipykernel_6431/2035906928.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[' 55.642371' ' 5.398574' ' -61.598425' ... ' 4.888678' ' 5.930466'
 ' 2.164508']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_silver.loc[condition_remplissage & df_silver['long'].isna(), 'long'] = coords_split[1]



Après amélioration (lat) : 15,206 valeurs manquantes (3.20 %).
Après amélioration (long): 15,202 valeurs manquantes (3.19 %).

✅ Succès ! Le DataFrame 'df_silver' en mémoire a été mis à jour.


In [ ]:
# =====================================================================
# DÉFINITION DES FONCTIONS DE CRÉATION DES DATAFRAMES
# =====================================================================
import pandas as pd

def creer_df_vehicule(df_source: pd.DataFrame) -> pd.DataFrame:
    """
    Crée le DataFrame des véhicules à partir du DataFrame source (df_silver).
    Nettoie les colonnes, crée une clé primaire 'id_veh', et dédoublonne.
    """
    print("=====================================================")
    print("        Fonction : Création de df_vehicule           ")
    print("=====================================================")
    
    # --- ÉTAPE 1 : Nettoyage Profond ---
    colonnes_a_nettoyer = ['num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    print("\nNettoyage profond des colonnes...")
    for colonne in colonnes_a_nettoyer:
        df_source[colonne] = df_source[colonne].astype(str).str.split(',').str[0].str.strip()
    print("-> Colonnes nettoyées.")

    # --- ÉTAPE 2 : Sélection et Création du DataFrame ---
    col_vehicule = ['num_acc', 'num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    df_vehicule = df_source[col_vehicule].copy()
    
    # --- ÉTAPE 3 : Application des Contraintes d'Unicité ---
    df_vehicule.dropna(subset=['num_acc', 'num_veh'], inplace=True)
    #df_vehicule.drop_duplicates(subset=['id_veh'], inplace=True)
    
    # --- ÉTAPE 4 : Finalisation du DataFrame ---
    colonnes_finales = ['num_acc', 'num_veh', 'place', 'choc', 'manv', 'catv', 'secu']
    df_vehicule = df_vehicule[colonnes_finales]
    
    print(f"\n✅ DataFrame 'df_vehicule' créé avec succès ({len(df_vehicule):,} lignes).")
    return df_vehicule


def creer_df_commune(df_source: pd.DataFrame) -> pd.DataFrame:
    """
    Crée le DataFrame des communes à partir du DataFrame source (df_silver).
    Sélectionne les colonnes pertinentes et dédoublonne par code commune.
    """
    print("\n\n=====================================================")
    print("         Fonction : Création de df_commune           ")
    print("=====================================================")
    
    # --- ÉTAPE 1 : Sélection des Colonnes ---

    col_commune = [
        'com', 'com_name', 'com_arm_name', 'epci_code','nom_com', 'adr',
        'lat', 'long', 'code_postal', 'coordonnees', 'code_com', 'num'
    ]

    df_commune = df_source[col_commune].copy()

    # --- ÉTAPE 2 : Dédoublonnage et Nettoyage ---
    # On dédoublonne sur la colonne 'com' qui semble être la clé la plus fiable ici
    df_commune.dropna(subset=['com'], inplace=True)
    df_commune.drop_duplicates(subset=['com'], inplace=True)
    
    # --- ÉTAPE 3 : Renommage ---
    df_commune.rename(columns={'code_com': 'com_code'}, inplace=True)

    print(f"\n✅ DataFrame 'df_commune' créé avec succès ({len(df_commune):,} lignes).")
    return df_commune


# =====================================================================
# ORCHESTRATION : APPEL DES FONCTIONS
# =====================================================================

# Prérequis : Le DataFrame 'df_silver' doit exister.
if 'df_silver' in locals() and df_silver is not None:
    # On appelle la première fonction pour créer df_vehicule
    df_vehicule = creer_df_vehicule(df_silver)
    
    # On appelle la seconde fonction pour créer df_commune
    df_commune = creer_df_commune(df_silver)
    
    # --- Validation Finale ---
    print("\n\n--- VALIDATION FINALE ---")
    print("Aperçu de df_vehicule :")
    print(df_vehicule.head())
    
    print("\nAperçu de df_commune :")
    print(df_commune.head())
    
else:
    print("❌ Erreur : Le DataFrame 'df_silver' n'a pas été trouvé. Veuillez exécuter les cellules précédentes.")

        Fonction : Création de df_vehicule           

Nettoyage profond des colonnes...
-> Colonnes nettoyées.

✅ DataFrame 'df_vehicule' créé avec succès (475,911 lignes).


         Fonction : Création de df_commune           

✅ DataFrame 'df_commune' créé avec succès (12,335 lignes).


--- VALIDATION FINALE ---
Aperçu de df_vehicule :
        num_acc num_veh place          choc                          manv  \
0  201600055393     A01     1         Avant  Sans changement de direction   
1  201600056285     A01     1  Avant gauche             Tournant A gauche   
2  201600056621     B01     1         Avant  Sans changement de direction   
3  201600057859     B02     3         Avant  Sans changement de direction   
4  201400013574     A01     1         Avant  Sans changement de direction   

                     catv                secu  
0                 VL seul              Casque  
1                 VL seul            Ceinture  
2  Motocyclette > 125 cm3              Casque  
3  